In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, LineString
from datetime import datetime

## Read shapefiles into variables

In [2]:
# Read shape files into geopandas
right_half_tri_SF = gpd.read_file("shapefiles/right_half_triangle/right_half_triangle.shp")[['Name', 'geometry']]
left_half_tri_SF = gpd.read_file("shapefiles/left_half_triangle/left_half_triangle.shp")[['Name', 'geometry']]
island_SF = gpd.read_file("shapefiles/island/island.shp")[['Name', 'geometry']]
newport_SF = gpd.read_file("shapefiles/inundation_map/inundation_map.shp")[['Name', 'geometry']]
# Read Street Shape Files
balboa_blvd = gpd.read_file("shapefiles/Streets/Balboa/Balboa_Blvd-polygon.shp")[['Name', 'geometry']]
w_balboa_blvd = gpd.read_file("shapefiles/Streets/W_Balboa/W_Balboa-polygon.shp")[['Name', 'geometry']]
newport = gpd.read_file("shapefiles/Streets/Newport/Newport_Blvd-polygon.shp")[['Name', 'geometry']]

In [3]:
section_shapefile_list = [right_half_tri_SF,left_half_tri_SF,island_SF,newport_SF]
street_shapefile_list = [balboa_blvd,w_balboa_blvd,newport]

## Read data for each day

In [4]:
# Assign data for each day
d_1_07012018 = 'Data/Probe_Data/2018_07_01_NewportBeach_basic_probe.csv'
d_2_07022018 = 'Data/Probe_Data/2018_07_02_NewportBeach_basic_probe.csv'
d_3_07032018 = 'Data/Probe_Data/2018_07_03_NewportBeach_basic_probe.csv'
d_4_07042018 = 'Data/Probe_Data/2018_07_04_NewportBeach_basic_probe.csv'
d_5_07052018 = 'Data/Probe_Data/2018_07_05_NewportBeach_basic_probe.csv'
d_6_07062018 = 'Data/Probe_Data/2018_07_06_NewportBeach_basic_probe.csv'
d_7_07072018 = 'Data/Probe_Data/2018_07_07_NewportBeach_basic_probe.csv'

In [5]:
data = pd.read_csv(d_4_07042018,
                   names = ["PROBE_ID","SAMPLE_DATE", "LAT", "LONG", "HEADING",\
                           "SPEED","PROBE_DATA_PROVIDER","X","Y", "LOCAL_TIME"])
# This now sorts in date order
data.sort_values(by='SAMPLE_DATE', inplace=True, ascending=True) 
data.head(3)

,PROBE_ID,SAMPLE_DATE,LAT,LONG,HEADING,SPEED,PROBE_DATA_PROVIDER,X,Y,LOCAL_TIME
480,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:00,33.615262,-117.927329,34.0,14.0,CONSUMER14,413977.562793,3.719884e+06,2018-07-03 17:00:00 -0700
479,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:01,33.615269,-117.927325,19.0,12.0,CONSUMER14,413977.940815,3.719885e+06,2018-07-03 17:00:01 -0700
445,285613c2-f71d-55f6-bb2b-8f16528c0a2c,2018-07-04 00:00:02,33.615287,-117.927314,19.0,17.0,CONSUMER14,413978.979134,3.719886e+06,2018-07-03 17:00:02 -0700


# Analyze Data

In [6]:
class Analyze_Data:
    
    def __init__(self, data_df,section_shapefile_list,street_shapefile_list):
        # Reading data dataframe into class
        self.data = data_df 
        
        # Read shapefiles into class as lists
        self.section = section_shapefile_list
        self.street = street_shapefile_list
        
    
    def plot(self):
        # Plot Data
        plt.figure(figsize = (12,8))
        plt.scatter(self.data.LONG, self.data.LAT, s = 0.5, c = 'k')
        plt.show()
    
    
    def create_point(self,row):
        # Helper function for turning coordinates into shapely points
        return Point(row['LONG'],row['LAT'])
    
    def append_points(self):

        self.data['coord'] = self.data.apply(self.create_point, axis=1)
        self.data['section'] = 'N/A'
        
        return self.data
        
    def get_section(self):
        # For separating data into sections based on defined shapefiles
        
        self.append_points()
        
        section_list = []
        
        for i in self.data.index.values:
            if self.section[0]['geometry'][0].contains(self.data['coord'][i]):
                section_list.append('right_half_tri')
            elif self.section[1]['geometry'][0].contains(self.data['coord'][i]):
                section_list.append('left_half_tri')
            elif self.section[2]['geometry'][0].contains(self.data['coord'][i]):
                section_list.append('island')
            elif self.section[3]['geometry'][0].contains(self.data['coord'][i]):
                section_list.append('rest_of_new_port')
            else:
                section_list.append('N/A')
        
        self.data['section'] = section_list
        
        return self.data
    
    def get_street(self):
        # For separating data into streets based on defined shapefiles
        
        self.get_section()
        street_list = []

        for i in self.data.index.values:
            if self.street[0]['geometry'][0].contains(self.data['coord'][i]):
                street_list.append('balboa')
            elif self.street[1]['geometry'][0].contains(self.data['coord'][i]):
                street_list.append('w_balboa')
            elif self.street[2]['geometry'][0].contains(self.data['coord'][i]):
                street_list.append('newport')
            else:
                street_list.append('N/A')

        self.data['street'] = street_list
        
        return self.data
    
    def clean_data(self):
        # Remove useless columns and set timezone
        self.get_street()
        self.data_cleaned = self.data[self.data.section != 'N/A'][['PROBE_ID','LAT','LONG',\
                                                            'HEADING',\
                                                            'SPEED',\
                                                            'PROBE_DATA_PROVIDER',\
                                                            'LOCAL_TIME',\
                                                            'coord',\
                                                            'section','street']]
        
        self.data_cleaned['LOCAL_TIME'] = pd.to_datetime(self.data_cleaned['LOCAL_TIME'])\
                                        .dt.tz_localize('UTC').dt.tz_convert('America/Los_Angeles')
        # Separate to hours
        self.data_cleaned['HOUR'] = self.data_cleaned['LOCAL_TIME'].dt.hour  
        
        return self.data_cleaned
    
    def clean_data_add_heading(self):
        # Add heading information per probe
        self.clean_data()
        
        heading_fixed = []
        
        for i in self.data_cleaned.index.values:
            if self.data_cleaned['HEADING'][i] >180:
                heading_fixed.append(-(360 - self.data_cleaned['HEADING'][i]))
            else:
                heading_fixed.append(self.data_cleaned['HEADING'][i])
        
        self.data_cleaned['HEADING_FIXED'] = heading_fixed
        
        return self.data_cleaned

    
    def probe_count(self, section):
        # For counting number of uniuqe probes
        self.clean_data_add_heading()
        uniq_probe = self.data_cleaned.loc[self.data_cleaned['section'] == section].PROBE_ID.unique()

        return len(uniq_probe)
    
    

In [7]:
class road_specific_analysis():
    
    def __init__(self, data_cleaned, street_choice):
        self.data = data_cleaned
        self.street = street_choice
    
    def road_df(self):
        # Filter and create dataframe based on road choice
        self.data
        self.road = self.data.loc[self.data['street']== self.street].copy()
        
        return self.road
    
    def get_heading(self, df, i, lower_head, upper_head):
        
        # Helper function for differentiating heading
        head_df = df.loc[(df['HOUR']==i) & \
                         ((df['HEADING_FIXED'] >= lower_head) | \
                         (df['HEADING_FIXED'] <= upper_head))][['PROBE_ID','SPEED']]
        
        # This removes all 0 speed instances
        head_df = head_df[head_df['SPEED'] != 0]
        unique_probe_list = list(set(head_df['PROBE_ID']))
        
        return head_df, unique_probe_list
    
    def get_heading_info(self, limits, speed_limit, trans_type):
        
        '''
        Inputs:
            df - cleaned up dataframe with hours and streets 
            limits - list of heading limits, [lower, higher]
            trans_type - looking for 'vehicle' or 'pedestrian'
            
        Outputs:
            head_speed - list of 24 north heading speeds averaged over an hour
            head_unique_probe - list of 24 counts of unique probe ids in that hour - vehicles
        '''
        
        self.road_df()
        
        head_speed = []
        head_unique_probe = []
        
        for i in range(24):
        # Analyze for 24 hours
            head_df, unique_probe_list = self.get_heading(self.road, i, limits[0], limits[1])
            
            count = 0
            speed = []
            
            for j in unique_probe_list:

                probe_df = head_df.loc[head_df['PROBE_ID'] == j]
                mean_speed = np.mean(probe_df['SPEED'])

                if trans_type == 'Vehicle':

                    if mean_speed >= speed_limit:
                        count += 1
                        speed.append(mean_speed)

                elif trans_type == 'Pedestrian':

                    if mean_speed <= speed_limit:
                        count += 1
                        speed.append(mean_speed)
                    
        
            head_unique_probe.append(count)

            if not speed:
                head_speed.append(0)
            else:
                head_speed.append(np.mean(speed))
            
        return head_speed, head_unique_probe
            

# For getting trajectory

In [8]:
traj = Analyze_Data(data, section_shapefile_list, street_shapefile_list)
traj_df = traj.clean_data_add_heading()
traj_df.head()

,PROBE_ID,LAT,LONG,HEADING,SPEED,PROBE_DATA_PROVIDER,LOCAL_TIME,coord,section,street,HOUR,HEADING_FIXED
480,285613c2-f71d-55f6-bb2b-8f16528c0a2c,33.615262,-117.927329,34.0,14.0,CONSUMER14,2018-07-03 17:00:00-07:00,POINT (-117.927329 33.615262),right_half_tri,N/A,17,34.0
479,285613c2-f71d-55f6-bb2b-8f16528c0a2c,33.615269,-117.927325,19.0,12.0,CONSUMER14,2018-07-03 17:00:01-07:00,POINT (-117.927325 33.615269),right_half_tri,N/A,17,19.0
445,285613c2-f71d-55f6-bb2b-8f16528c0a2c,33.615287,-117.927314,19.0,17.0,CONSUMER14,2018-07-03 17:00:02-07:00,POINT (-117.927314 33.615287),right_half_tri,N/A,17,19.0
494,6c247e2bf8f447b2b31b7450261a1e3c,33.618990,-117.930160,189.0,4.0,CONSUMER21,2018-07-03 17:00:03-07:00,POINT (-117.93016 33.61899),right_half_tri,newport,17,-171.0
2276,285613c2-f71d-55f6-bb2b-8f16528c0a2c,33.615319,-117.927299,18.0,20.0,CONSUMER14,2018-07-03 17:00:03-07:00,POINT (-117.927299 33.615319),right_half_tri,N/A,17,18.0


In [9]:
uniq_ID = set(traj_df.PROBE_ID)

# Create empty dataframe
columns = ['PROBE_ID','Route_Num','Start_Section','End_Section','Time_Start','Time_End']
traj_route_df = pd.DataFrame(columns=columns)

for ID in uniq_ID:
    
    probe_df = traj_df.loc[traj_df.PROBE_ID == ID]
    
    route = [probe_df.section.iloc[0]]
    route_count = 0
    route_time = [probe_df.LOCAL_TIME.iloc[0]]
    
    for i in range(len(probe_df)-1):

        if (probe_df.LOCAL_TIME[i+1] - probe_df.LOCAL_TIME[i])<pd.Timedelta(minutes = 1):

            route.append(probe_df.street.iloc[i+1])
            route_time.append(probe_df.LOCAL_TIME.iloc[i+1])

        else:
            route_count += 1  
            traj_route_df = traj_route_df.append({'PROBE_ID':ID,'Route_Num':route_count,\
                                                  'Start_Section':route[0],'End_Section':route[-1],\
                                                'Time_Start':route_time[0],'Time_End':route_time[-1]},\
                                                ignore_index=True)
            



In [10]:
traj_rest = traj_route_df.loc[traj_route_df['Start_Section'] == 'rest_of_new_port']

In [11]:
balboa = traj_rest.loc[traj_rest['End_Section'] == 'balboa']
newport = traj_rest.loc[traj_rest['End_Section'] == 'newport']

In [12]:
balboa = balboa.drop_duplicates(subset = ['Time_Start','Time_End'],keep=False)
newport = newport.drop_duplicates(subset = ['Time_Start','Time_End'],keep=False)

In [13]:
balboa

,PROBE_ID,Route_Num,Start_Section,End_Section,Time_Start,Time_End
373,c9dae7a716114ece8f0395e27ab88668,17,rest_of_new_port,balboa,2018-07-03 17:00:04-07:00,2018-07-03 17:36:34-07:00
374,c9dae7a716114ece8f0395e27ab88668,18,rest_of_new_port,balboa,2018-07-03 17:00:04-07:00,2018-07-03 17:37:49-07:00
375,c9dae7a716114ece8f0395e27ab88668,19,rest_of_new_port,balboa,2018-07-03 17:00:04-07:00,2018-07-03 17:38:04-07:00
924,5636e01b-b74a-5484-bea6-a302424e5551,1,rest_of_new_port,balboa,2018-07-03 17:00:06-07:00,2018-07-03 17:02:02-07:00
925,5636e01b-b74a-5484-bea6-a302424e5551,2,rest_of_new_port,balboa,2018-07-03 17:00:06-07:00,2018-07-03 17:02:39-07:00
926,5636e01b-b74a-5484-bea6-a302424e5551,3,rest_of_new_port,balboa,2018-07-03 17:00:06-07:00,2018-07-03 17:04:23-07:00
927,5636e01b-b74a-5484-bea6-a302424e5551,4,rest_of_new_port,balboa,2018-07-03 17:00:06-07:00,2018-07-03 17:05:10-07:00


In [14]:
newport

,PROBE_ID,Route_Num,Start_Section,End_Section,Time_Start,Time_End
443,0C023DA300,36,rest_of_new_port,newport,2018-07-04 10:50:04-07:00,2018-07-04 14:25:27-07:00
1006,57b40fd4696c48cfb3d46c786902d36e,4,rest_of_new_port,newport,2018-07-04 16:23:35-07:00,2018-07-04 16:49:26-07:00
1058,8c2c552aa42a40e68b4ca64f4bef9308,13,rest_of_new_port,newport,2018-07-03 17:56:11-07:00,2018-07-03 20:33:57-07:00


In [15]:
print('Number of Cars going to Balboa Blvd:', len(balboa))
print('Number of Cars going to Newport Blvd:', len(newport))

Number of Cars going to Balboa Blvd: 7
Number of Cars going to Newport Blvd: 3


## Notes
### For unique probe count
### Section Options:
- 'right_half_tri'
- 'left_half_tri'
- 'island'
- 'rest_of_new_port'

### Street Options
- 'balboa'
- 'newport'


### Blanket sorting for all data

In [16]:
analysis = Analyze_Data(data, section_shapefile_list, street_shapefile_list)
data_cleaned = analysis.clean_data_add_heading()

In [17]:
data_cleaned.head()

,PROBE_ID,LAT,LONG,HEADING,SPEED,PROBE_DATA_PROVIDER,LOCAL_TIME,coord,section,street,HOUR,HEADING_FIXED
480,285613c2-f71d-55f6-bb2b-8f16528c0a2c,33.615262,-117.927329,34.0,14.0,CONSUMER14,2018-07-03 17:00:00-07:00,POINT (-117.927329 33.615262),right_half_tri,N/A,17,34.0
479,285613c2-f71d-55f6-bb2b-8f16528c0a2c,33.615269,-117.927325,19.0,12.0,CONSUMER14,2018-07-03 17:00:01-07:00,POINT (-117.927325 33.615269),right_half_tri,N/A,17,19.0
445,285613c2-f71d-55f6-bb2b-8f16528c0a2c,33.615287,-117.927314,19.0,17.0,CONSUMER14,2018-07-03 17:00:02-07:00,POINT (-117.927314 33.615287),right_half_tri,N/A,17,19.0
494,6c247e2bf8f447b2b31b7450261a1e3c,33.618990,-117.930160,189.0,4.0,CONSUMER21,2018-07-03 17:00:03-07:00,POINT (-117.93016 33.61899),right_half_tri,newport,17,-171.0
2276,285613c2-f71d-55f6-bb2b-8f16528c0a2c,33.615319,-117.927299,18.0,20.0,CONSUMER14,2018-07-03 17:00:03-07:00,POINT (-117.927299 33.615319),right_half_tri,N/A,17,18.0


### Analysis of data based on road and transportation type

In [18]:
# Balboa
bal_north_limits = [-90,45]
bal_south_limits = [145,-150]
# Newport
newport_north_limits = [-140,0]  
newport_south_limits = [90,-160]

speed_limit = 15

In [19]:
# For Balboa
analysis_balboa = road_specific_analysis(data_cleaned,'balboa')

# Northbound
veh_balboa_N_speed, veh_balboa_N_probe = analysis_balboa.\
                                         get_heading_info(bal_north_limits,\
                                         speed_limit, "Vehicle")


ped_balboa_N_speed, ped_balboa_N_probe = analysis_balboa.\
                                         get_heading_info(bal_north_limits,\
                                         speed_limit, "Pedestrian")


# Southbound

veh_balboa_S_speed, veh_balboa_S_probe = analysis_balboa.\
                                         get_heading_info(bal_south_limits,\
                                         speed_limit, "Vehicle")


ped_balboa_S_speed, ped_balboa_S_probe = analysis_balboa.\
                                         get_heading_info(bal_south_limits,\
                                         speed_limit, "Pedestrian")


In [20]:
# For Newport
analysis_newport = road_specific_analysis(data_cleaned,'newport')

# Northbound
veh_newport_N_speed, veh_newport_N_probe = analysis_newport.\
                                         get_heading_info(newport_north_limits,\
                                         speed_limit, "Vehicle")


ped_newport_N_speed, ped_newport_N_probe = analysis_newport.\
                                         get_heading_info(newport_north_limits,\
                                         speed_limit, "Pedestrian")


# Southbound

veh_newport_S_speed, veh_newport_S_probe = analysis_newport.\
                                         get_heading_info(newport_south_limits,\
                                         speed_limit, "Vehicle")


ped_newport_S_speed, ped_newport_S_probe = analysis_newport.\
                                         get_heading_info(newport_south_limits,\
                                         speed_limit, "Pedestrian")

In [21]:
# Create a dataframe of road information per hour
road_info_df = pd.DataFrame({'Hour':range(0,24),\
                'Newport_North_Speed_Veh':veh_newport_N_speed,\
                'Newport_South_Speed_Veh':veh_newport_S_speed,\
                'Newport_North_Probe_Veh':veh_newport_N_probe,\
                'Newport_South_Probe_Veh':veh_newport_S_probe,\
                'Newport_North_Speed_Ped':ped_newport_N_speed,\
                'Newport_South_Speed_Ped':ped_newport_S_speed,\
                'Newport_North_Probe_Ped':ped_newport_N_probe,\
                'Newport_South_Probe_Ped':ped_newport_S_probe,\
                             
                'Balboa_North_Speed_Veh':veh_balboa_N_speed,\
                'Balboa_South_Speed_Veh':veh_balboa_S_speed,\
                'Balboa_North_Probe_Veh':veh_balboa_N_probe,\
                'Balboa_South_Probe_Veh':veh_balboa_S_probe,\
                'Balboa_North_Speed_Ped':ped_balboa_N_speed,\
                'Balboa_South_Speed_Ped':ped_balboa_S_speed,\
                'Balboa_North_Probe_Ped':ped_balboa_N_probe,\
                'Balboa_South_Probe_Ped':ped_balboa_S_probe})




In [22]:
road_info_df.head()

,Hour,Newport_North_Speed_Veh,Newport_South_Speed_Veh,Newport_North_Probe_Veh,Newport_South_Probe_Veh,Newport_North_Speed_Ped,Newport_South_Speed_Ped,Newport_North_Probe_Ped,Newport_South_Probe_Ped,Balboa_North_Speed_Veh,Balboa_South_Speed_Veh,Balboa_North_Probe_Veh,Balboa_South_Probe_Veh,Balboa_North_Speed_Ped,Balboa_South_Speed_Ped,Balboa_North_Probe_Ped,Balboa_South_Probe_Ped
0,0,41.329114,45.329114,2,2,0.000000,0.000000,0,0,32.8125,25.777778,1,1,0.000000,0.0,0,0
1,1,27.358780,27.358780,1,1,0.000000,0.000000,0,0,0.0000,0.000000,0,0,0.000000,0.0,0,0
2,2,27.285714,15.571429,2,1,0.000000,0.000000,0,0,0.0000,0.000000,0,0,5.734340,7.5,2,1
3,3,30.577460,30.577460,1,1,10.720389,8.966323,2,1,0.0000,0.000000,0,0,10.856683,0.0,2,0
4,4,41.038170,15.288730,1,1,11.905691,0.000000,2,0,0.0000,0.000000,0,0,0.000000,0.0,0,0


In [23]:


def find_slowest(df, column,probe):
    
    slow = []
    for i in df[column]:
        if i>0:
            slow.append(i)
    slowest_speed = min(slow)
    slowest_df = df.loc[df[column]==slowest_speed][['Hour', column, probe]]

    return slowest_df



In [24]:
N_N_slow = find_slowest(road_info_df,'Newport_North_Speed_Veh','Newport_North_Probe_Veh')
S_N_slow = find_slowest(road_info_df,'Newport_South_Speed_Veh','Newport_South_Probe_Veh')

N_B_slow = find_slowest(road_info_df,'Balboa_North_Speed_Veh','Balboa_North_Probe_Veh')
S_B_slow = find_slowest(road_info_df,'Balboa_South_Speed_Veh','Balboa_South_Probe_Veh')

In [25]:
N_N_slow

,Hour,Newport_North_Speed_Veh,Newport_North_Probe_Veh
14,14,17.756836,3


In [26]:
S_N_slow

,Hour,Newport_South_Speed_Veh,Newport_South_Probe_Veh
14,14,15.020507,1


In [27]:
N_B_slow

,Hour,Balboa_North_Speed_Veh,Balboa_North_Probe_Veh
7,7,17.083333,2


In [28]:
S_B_slow

,Hour,Balboa_South_Speed_Veh,Balboa_South_Probe_Veh
19,19,16.25,2
